In [1]:
import sklearn
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
from collections import Counter
import io
#from google.colab import files   
#uploaded = files.upload()
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import ExtraTreesClassifier


In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[265]:




# In[152]:

def train_model():

    data = pd.read_csv('sepsis.csv')

    # In[153]:

    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy="median")
    imputer.fit(data)
    x= imputer.transform(data)

    # In[154]:


    #from sklearn import preprocessing
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(data)
    data = pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)


    # In[171]:


    x = data.loc[:, data.columns!= 'SepsisLabel']
    y = data.loc[:, data.columns == 'SepsisLabel']
    #type(x)


    # In[221]:


    #from sklearn.decomposition import PCA

    pca = PCA(n_components=20)

    x_pca = pca.fit_transform(x)
    #scores=pd.DataFrame(x_pca)
    #print(scores)
    explained_variance = pca.explained_variance_ratio_
    #print(pd.DataFrame(pca.components_,columns=x.columns))
    #explained_variance
    #s=pca.components_
    #s.shape
    #x_pca.shape



    # In[175]:


    #from sklearn.model_selection import train_test_split, StratifiedKFold
    X_train, X_test, y_train, y_test = train_test_split(x_pca,y,test_size=0.25, random_state = 62)


    # In[177]:


    #!pip install -U imbalanced-learn
    #import imblearn
    #from imblearn.over_sampling import SMOTE


    # In[273]:


    #%%time
    kf = StratifiedKFold(n_splits=4, random_state=62)
    cross_val_f1_score_lst = []
    cross_val_accuracy_lst = []
    cross_val_recall_lst = []
    cross_val_precision_lst = []


    for train_index_ls, validation_index_ls in kf.split(X_train, y_train):
        # keeping validation set apart and oversampling in each iteration using smote 
        train, validation = X_train[train_index_ls], X_train[validation_index_ls]
        target_train, target_val = y_train.iloc[train_index_ls], y_train.iloc[validation_index_ls]
        sm = SMOTE(random_state=62)
        X_train_res, y_train_res = sm.fit_sample(train, target_train)
        #print (X_train_res.shape, y_train_res.shape)
        
       #ExtraTreeClassifier
        #from sklearn.ensemble import ExtraTreesClassifier
        etc = ExtraTreesClassifier( n_estimators=1000, criterion="entropy", max_features="auto", min_samples_leaf=1, min_samples_split=5)
        
        # fit the model on the whole dataset
        etc.fit(X_train_res, y_train_res)
        
        #predict on the train data
        etc_predict = etc.predict(validation)
        
        '''
        from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score,confusion_matrix
        cross_val_recall_lst.append(recall_score(target_val, etc_predict))
        cross_val_accuracy_lst.append(accuracy_score(target_val, etc_predict))
        cross_val_precision_lst.append(precision_score(target_val, etc_predict))
        cross_val_f1_score_lst.append(f1_score(target_val, etc_predict))
    print ('Cross validated accuracy: {}'.format(np.mean(cross_val_accuracy_lst)))
    print ('Cross validated recall score: {}'.format(np.mean(cross_val_recall_lst)))
    print ('Cross validated precision score: {}'.format(np.mean(cross_val_precision_lst)))
    print ('Cross validated f1_score: {}'.format(np.mean(cross_val_f1_score_lst)))
    '''


    # In[248]:



    #predict on the test data
    etc_predict_test = etc.predict(X_test)
    #print(X_test)
    #print(etc_predict_test)
    #print(type(X_test))
    #print(len(X_test))

    '''
    from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score,confusion_matrix
    print("accuracy:",accuracy_score(etc_predict_test,y_test))
    print("recall score:",recall_score(etc_predict_test,y_test))
    print("precision score: ", precision_score(etc_predict_test,y_test))
    print("f1 score: ", f1_score(etc_predict_test,y_test))
    print('train-set confusion matrix:\n', confusion_matrix(etc_predict_test,y_test))
    etc_list=[accuracy_score(etc_predict_test,y_test),recall_score(etc_predict_test,y_test),precision_score(etc_predict_test,y_test),f1_score(etc_predict_test,y_test)]
    print(etc_list)
    '''

    return pca,etc




In [3]:
# -*- coding: utf-8 -*-
"""module 2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1APfAIISuA27ITPKEi1QtidTc3S56f5_C
"""

# SIRS score function 

def sirs_score(temp, hr, rr, wbc):
  dict = {}
  sirs = 0
  if(temp>38):
    sirs=sirs+1
    dict['temp'] = 'high'
  elif(temp<36):
    dict['temp'] = 'low'
    sirs=sirs+1
  else:
    dict['temp'] = 'normal'
  
  if(hr>90):
    dict['hr'] = 'high'
    sirs=sirs+1
  else:
    dict['hr'] = 'normal'

  if(rr>20):
    dict['rr'] = 'high'
    sirs=sirs+1
  else:
    dict['rr'] = 'normal'

  if(wbc>12):
    sirs=sirs+1
    dict['wbc'] = 'high'
  elif(wbc<4):
    dict['wbc'] = 'low'
    sirs=sirs+1
  else:
    dict['wbc'] = 'normal'

  return sirs,dict

##sample input
##sirs_score(36.11,108,29,5.7)



In [4]:
# -*- coding: utf-8 -*-
"""module3.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1atRFuOoVaGKCGznKtHFU-MvonjmVxe1n
"""

# Module 3 - medical recomendation

def medical_rec(sympt):
    rec={}
    hr_high=["Diuretics", "Beta-blockers", "ACE inhibitors", "Vasodilators", "Angiotensin II receptor blockers"]
    temp_high=["Acetaminophen", "Ibuprofen", "Aspirin"]
    wbc_high=["Antibiotics", "Anticonvulsants", "Antithyroid drugs","Arsenicals","Captopril","Chlorpromazine","Clozapine"]
    wbc_low=["Beta-adrenergic agonist","Corticosteroids","Epinephrine","Granulocyte colony-stimulating factor","Heparin","Lithium"]
    rr_high=["alprazolam", "clonazepam", "buspirone"]

    if(sympt['temp']=='high'):
        rec['high_temp']=temp_high
    
    if(sympt['rr']=='high'):
        rec['high_rr']=rr_high;
    
    if(sympt['hr']=='high'):
        rec['high_hr']=hr_high
    
    if(sympt['wbc']=='low'):
        rec['low_wbc']= wbc_low;
    elif(sympt['wbc']=='high'):
      rec['high_wbc']= wbc_high

    return rec;

#sample input
##medical_rec({'hr': 'high', 'rr': 'high', 'temp': 'normal', 'wbc': 'normal'})



In [5]:

def query_input(pca,etc):

    attribute_list=['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
           'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
           'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
           'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
           'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
           'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
           'HospAdmTime', 'ICULOS']

    # In[243]:

    attribute_dict = dict.fromkeys(attribute_list , 0)

    attribute_dict['HR']=119
    attribute_dict['O2Sat']=100
    attribute_dict['Temp']=37.94
    attribute_dict['SBP']=140
    attribute_dict['MAP']=106
    attribute_dict['DBP']=85
    attribute_dict['Resp']=26.5
    attribute_dict['FiO2']=0.35
    attribute_dict['pH']=0
    attribute_dict['SaO2']=0
    attribute_dict['BUN']=0
    attribute_dict['Calcium']=0
    attribute_dict['Chloride']=0
    attribute_dict['Glucose']=0
    attribute_dict['Hgb']=0
    attribute_dict['WBC']=189
    attribute_dict['Age']=27.92
    attribute_dict['Gender']=1
    attribute_dict['Unit1']=0
    attribute_dict['Unit2']=0
    attribute_dict['HospAdmTime']=-0.03
    attribute_dict['ICULOS']=249
    #attribute_dict

    # In[264]:


    #from sklearn.preprocessing import MinMaxScaler

    user_input=pd.DataFrame.from_dict(attribute_dict,orient='index')      #<class 'pandas.core.frame.DataFrame'>
    #print(type(user_input)) #dataframe

    scalar = preprocessing.MinMaxScaler()
    scaled_data=scalar.fit_transform(user_input)             #<class 'numpy.ndarray'>
    #print(scaled_data)
    #print(type(scaled_data))#numpyarray

    temp = pd.DataFrame(scaled_data,index=user_input.index,columns=user_input.columns)
    input_data=temp.transpose()                               #<class 'pandas.core.frame.DataFrame'>
    #print(input_data)
    #print(type(input_data)) #dataframe
    red_data_pca=pca.transform(input_data)                      #<class 'numpy.ndarray'>
    #print(type(red_data_pca)) #numpy array
    #print(red_data_pca.shape)
    #print(red_data_pca)
   
    New_predict = etc.predict(red_data_pca)

    print(int(New_predict[0]))


    if(int(New_predict[0])):
        score,dictionary=sirs_score(attribute_dict['Temp'], attribute_dict['HR'], attribute_dict['Resp'], attribute_dict['WBC'])
        if(score<=2):
            print("Sepsis Stage 1")
        if(score==3):
            print("Sepsis Stage 2")
        if(score==4):
            print("Sepsis Stage 3")
        medication_dict=medical_rec(dictionary)
        print("Medical recommendation")
        print(medication_dict)


In [6]:
pca_var,etc_var=train_model()

C:\Users\user\anaconda33\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
<ipython-input-2-ccb3dbe927a7>:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  etc.fit(X_train_res, y_train_res)
<ipython-input-2-ccb3dbe927a7>:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  etc.fit(X_train_res, y_train_res)
<ipython-input-2-ccb3dbe927a7>:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  etc.fit(X_train_res, y_train_res)
<ipython-input-2-ccb3dbe927a7>:97: Data

In [7]:
query_input(pca_var,etc_var)

0
